In [18]:
import optuna
import gc
import re
import json
import requests
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import random
import argparse
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import sys
from tqdm import tqdm
from collections import Counter
from collections import defaultdict
from logging import getLogger
from ast import Global

sys.path.append('/root/linghui/drs/TRD-main/daisyRec/')
import daisy
import imp

from daisy.utils.loader import RawDataReader, Preprocessor
from daisy.utils.splitter import TestSplitter, ValidationSplitter
from daisy.utils.config import init_seed, init_config, init_logger
from daisy.utils.metrics import MAP, NDCG, Recall, Precision, HR, MRR
from daisy.utils.sampler import BasicNegtiveSampler, SkipGramNegativeSampler, UniqueNegativeSampler
from daisy.utils.dataset import AEDataset, BasicDataset, CandidatesDataset, get_dataloader
from daisy.utils.utils import get_history_matrix, get_ur, build_candidates_set, ensure_dir, get_inter_matrix

from daisy.model.MFRecommender import MF
from daisy.model.FMRecommender import FM
from daisy.model.NFMRecommender import NFM
from daisy.model.NGCFRecommender import NGCF
from daisy.model.EASERecommender import EASE
from daisy.model.SLiMRecommender import SLiM
from daisy.model.VAECFRecommender import VAECF
from daisy.model.NeuMFRecommender import NeuMF
from daisy.model.PopRecommender import MostPop
from daisy.model.KNNCFRecommender import ItemKNNCF
from daisy.model.PureSVDRecommender import PureSVD
from daisy.model.Item2VecRecommender import Item2Vec
from daisy.model.LightGCNRecommender import LightGCN
from daisy.utils.metrics import calc_ranking_results


In [19]:
model_config = {
    'mostpop': MostPop,
    'slim': SLiM,
    'itemknn': ItemKNNCF,
    'puresvd': PureSVD,
    'mf': MF,
    'fm': FM,
    'ngcf': NGCF,
    'neumf': NeuMF,
    'nfm': NFM,
    'multi-vae': VAECF,
    'item2vec': Item2Vec,
    'ease': EASE,
    'lightgcn': LightGCN,
}

metrics_config = {
    "recall": Recall,
    "mrr": MRR,
    "ndcg": NDCG,
    "hr": HR,
    "map": MAP,
    "precision": Precision,
}

tune_params_config = {
    'mostpop': [],
    'itemknn': ['maxk'],
    'puresvd': ['factors'],
    'slim': ['alpha', 'elastic'],
    'mf': ['num_ng', 'factors', 'lr', 'batch_size', 'reg_1', 'reg_2'],
    'fm': ['num_ng', 'factors', 'lr', 'batch_size', 'reg_1', 'reg_2'],
    'neumf': ['num_ng', 'factors', 'num_layers', 'dropout', 'lr', 'batch_size', 'reg_1', 'reg_2'],
    'nfm': ['num_ng', 'factors', 'num_layers', 'dropout', 'lr', 'batch_size', 'reg_1', 'reg_2'],
    'ngcf': ['num_ng', 'factors', 'node_dropout', 'mess_dropout', 'batch_size', 'lr', 'reg_1', 'reg_2'],
    'multi-vae': ['latent_dim', 'dropout','batch_size', 'lr', 'anneal_cap'],
    'ease': ['reg'],
    'item2vec': ['context_window', 'rho', 'lr', 'factors'],
    'lightgcn': ['num_ng', 'factors', 'batch_size', 'lr', 'reg_1', 'reg_2', 'num_layers'],
}

param_type_config = {
    'num_layers': 'int',
    'maxk': 'int',
    'factors': 'int',
    'alpha': 'float',
    'elastic': 'float',
    'num_ng': 'int',
    'lr': 'float',
    'batch_size': 'int',
    'reg_1': 'float',
    'reg_2': 'float',
    'dropout': 'float',
    'node_dropout': 'float',
    'mess_dropout': 'float',
    'latent_dim': 'int',
    'anneal_cap': 'float',
    'reg': 'float',
    'context_window': 'int',
    'rho': 'float'
}

In [35]:
config = init_config()
init_seed(config['seed'], config['reproducibility'])
init_logger(config)

In [36]:
config['debias_method'] = 'cnif'
config['weight'] = 10

logger = getLogger()
logger.info(config)
config['logger'] = logger


16 Jan 10:22 INFO - {'version': 'daisy_1/', 'gpu': '0', 'seed': 2024, 'reproducibility': True, 'state': None, 'optimization_metric': 'ndcg', 'hyperopt_trail': 20, 'tune_testset': False, 'tune_pack': '{"lr": [0.001, 0.005, 0.01], "reg_1": [0.000001, 0.00001, 0.0001, 0.001, 0], "reg_2": [0.000001, 0.00001, 0.0001, 0.001, 0], "num_layers": [2, 3, 4]}', 'algo_name': 'mf', 'data_path': '/data/linghui/', 'save_path': '/data/linghui/drs/', 'res_path': None, 'dataset': 'steam', 'val_method': 'tsbr', 'test_method': 'tsbr', 'fold_num': 1, 'val_size': 0.1, 'test_size': 0.2, 'topk': 50, 'cand_num': 1000, 'sample_method': 'uniform', 'sample_ratio': 0, 'num_ng': 4, 'batch_size': 1024, 'loss_type': 'BPR', 'init_method': 'default', 'optimizer': 'default', 'early_stop': True, 'prepro': '10filter', 'level': 'u', 'positive_threshold': 1.0, 'UID_NAME': 'user', 'IID_NAME': 'item', 'INTER_NAME': 'label', 'TID_NAME': 'timestamp', 'binary_inter': True, 'metrics': ['recall', 'mrr', 'ndcg', 'hit', 'precision'],

In [21]:
config['weight']

10

In [37]:
save_path = config['save_path'] + config['version']
ensure_dir(save_path)

file_path = save_path + f'{config["dataset"]}/'
ensure_dir(file_path)

saved_data_path = file_path + 'data/'
ensure_dir(saved_data_path)

saved_result_path = file_path + f'{config["algo_name"]}/'
ensure_dir(saved_result_path)

saved_model_path = saved_result_path + 'model/'
ensure_dir(saved_model_path)

saved_rec_path = saved_result_path + 'rec_list/'
ensure_dir(saved_rec_path)

saved_metric_path = saved_result_path + 'metric/'
ensure_dir(saved_metric_path)

In [39]:
ui_num = np.load(saved_data_path + 'ui_cate.npy')
# config['user_num'] = 1411
# config['item_num'] = 3317
config['user_num'] = ui_num[0]
config['item_num'] = ui_num[1]
print(f"user number: {config['user_num']}  item number: {config['item_num']}")
# config['cate_num'] = ui_num[2]
# print(f"user number: {config['user_num']}  item number: {config['item_num']}  cate number: {config['cate_num']}")

train_base_set = pd.read_csv(saved_data_path + 'train_base_set.csv', index_col=0)
# train_base_samples = np.load(saved_data_path + 'train_base_samples.npy')

train_total_set = pd.read_csv(saved_data_path + 'train_total_set.csv', index_col=0)
train_total_samples = np.load(saved_data_path + 'train_total_samples.npy')

user number: 79216  item number: 14258


/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
train_total_samples[:5]

array([[1221,    0,  941],
       [1221,    0,  922],
       [1221,    0, 1809],
       [1221,    0,  596],
       [1221,    1,  265]], dtype=int32)

In [28]:
train_set = train_total_set
train_total_ur = get_ur(train_total_set)
config['train_ur'] = train_total_ur
train_samples = train_total_samples
s_time = time.time()
if config['algo_name'].lower() in ['itemknn', 'puresvd', 'slim', 'mostpop', 'ease']:
    model = model_config[config['algo_name']](config)
    # model.fit(train_set)

elif config['algo_name'].lower() in ['mf', 'fm', 'neumf', 'nfm', 'ngcf', 'lightgcn']:
    if config['algo_name'].lower() in ['lightgcn', 'ngcf']:
        config['inter_matrix'] = get_inter_matrix(train_set, config)
    model = model_config[config['algo_name']](config)
    train_dataset = BasicDataset(train_samples)
    train_loader = get_dataloader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4)
    # model.fit(train_loader)

In [29]:
# def cgf_topk_loss(model_cgf, users, items, warm_items, cold_items, top_score):
#     item_num = len(items)
#     warm_nums = len(warm_items)
#     cold_nums = len(cold_items)
    
#     # 将数据移动到 GPU
#     model_cgf.cuda()
#     users = torch.tensor(users).cuda()
#     items = torch.tensor(items).cuda()
    
#     # 获取用户和物品的嵌入
#     user_embed = model_cgf.embed_user(users)
#     item_embed = model_cgf.embed_item(items)
    
#     # 计算分数矩阵
#     score_mat = torch.mm(item_embed, user_embed.t())
    
#     # 获取 top-k 分数和索引
#     _, item_index = score_mat.topk(top_score, dim=0)
    
#     # 使用掩码操作替代 scatter_ 和 gather
#     mask = torch.zeros_like(score_mat).cuda()
#     mask.scatter_(0, item_index, 1)  # 创建一个掩码，标记 top-k 的位置
#     score_top = score_mat * mask  # 通过掩码提取 top-k 分数
    
#     # 计算期望分数（使用对数归一化）
#     exp_item_list = score_top.mean(dim=1)
#     exp_item_list = exp_item_list / exp_item_list.sum()
    
#     # 批量计算权重
#     weight_list = torch.zeros(item_num).cuda()
#     weight_list[:warm_nums] = (warm_nums - torch.arange(warm_nums).cuda()) / warm_nums
#     weight_list[warm_nums:] = -(1 + (torch.arange(cold_nums).cuda() * (warm_nums - 1) / (cold_nums - 1))) / cold_nums
    
#     # 计算损失
#     cgf_loss = (exp_item_list * weight_list).sum()
    
#     return cgf_loss

def cgf_topk_loss(model_cgf, users, items, config):
    warm_items = config['warm_item_list']
    cold_items = config['cold_item_list']
    item_num = len(items)
    warm_nums = len(warm_items)
    cold_nums = len(cold_items)
    top_score = config['topk']
    print(f'all item num:{item_num}, warm_num:{warm_nums}, cold_num:{cold_nums}')
    
    # 将数据移动到 GPU
    model_cgf.cuda()
    users = torch.tensor(users).cuda()
    items = torch.tensor(items).cuda()
    
    # 获取用户和物品的嵌入
    if config['algo_name'] == 'mf':
        user_embed = model_cgf.embed_user(users)
        item_embed = model_cgf.embed_item(items)
    elif config['algo_name'] == 'lightgcn':
        user_emb_all, item_emb_all = model.forward()
        user_emb = user_emb_all[users]
        item_emb = item_emb_all[items]
    
    # 计算分数矩阵
    score_mat = torch.mm(item_embed, user_embed.t())
    # score_mat = F.softmax(score_mat, dim=0)
    
    # 获取 top-k 分数和索引
    _, item_index = score_mat.topk(top_score, dim=0)
    
    # 使用掩码操作替代 scatter_ 和 gather
    mask = torch.zeros_like(score_mat).cuda()
    mask.scatter_(0, item_index, 1)  # 创建一个掩码，标记 top-k 的位置
    score_top = score_mat * mask  # 通过掩码提取 top-k 分数
    
    # 计算期望分数（使用对数归一化）
    exp_item_list = score_top.mean(dim=1)
    exp_item_list = exp_item_list / exp_item_list.sum()
    
    # 批量计算权重
    weight_list = torch.zeros(item_num).cuda()
    weight_list[:warm_nums] = (warm_nums - torch.arange(warm_nums).cuda()) / warm_nums
    weight_list[warm_nums:] = -(1 + (torch.arange(cold_nums).cuda() * (warm_nums - 1) / (cold_nums - 1))) / cold_nums
    
    # 计算损失
    cgf_loss = (exp_item_list * weight_list).sum()
    
    return cgf_loss

In [30]:
def pcc_train(model_here, train_data, item_pop_dict, config):
    data2 = train_data.copy()
    
    data2['user'] = data2['user'].apply(lambda x : int(x))
    data2['item'] = data2['item'].apply(lambda x : int(x))    
        
    filter_users = data2.user.value_counts()[data2.user.value_counts() > 1].index
    data2 = data2[data2.user.isin(filter_users)]
    data2 = data2.reset_index()[['user', 'item']]
    
    user_num = len(data2.user.unique())
    data_len = data2.shape[0]
    frac = 50
    frac_user_num = int(data_len/frac)
    
    predictions_list = torch.tensor([]).cuda()
    
    model_here.cuda()
    
    for itr in range(frac):
        tmp = data2.iloc[ (frac_user_num*itr) : (frac_user_num*(itr+1)) ].values    
        user = tmp[:, 0]
        user = user.astype(np.int32)        
        user = torch.from_numpy(user).cuda()
        item = tmp[:, 1]
        item = item.astype(np.int32)        
        item = torch.from_numpy(item).cuda()
        if config['algo_name'] == 'mf':
            predictions = model_here(user, item)
        elif config['algo_name'] == 'lightgcn':
            embed_user = model.embed_user(user)
            embed_item = model.embed_item(item)
            predictions = (embed_user * embed_item).sum(dim=-1)
        
        predictions_list = torch.hstack((predictions_list, predictions))

        
        if itr+1 == frac:
            tmp = data2.iloc[ (frac_user_num*(itr+1)):].values    
            user = tmp[:, 0]
            user = user.astype(np.int32)        
            user = torch.from_numpy(user).cuda()
            item = tmp[:, 1]
            item = item.astype(np.int32)        
            item = torch.from_numpy(item).cuda()

            if config['algo_name'] == 'mf':
                predictions = model_here(user, item)
            elif config['algo_name'] == 'lightgcn':
                embed_user = model.embed_user(user)
                embed_item = model.embed_item(item)
                predictions = (embed_user * embed_item).sum(dim=-1)
            predictions_list = torch.hstack((predictions_list, predictions))

    # item_pop_dict = dict(item_pop.values)
    data2['item_pop_count'] = data2['item'].map(item_pop_dict)            
        
    values = predictions_list.reshape(-1, 1)
    item_pop_count = data2.item_pop_count.values
    item_pop_count = item_pop_count.astype(np.int32)
    item_pop_count = torch.from_numpy(item_pop_count).float().cuda()
    
    
    X = values
    Y = item_pop_count # item pop
    
    # print('size of X: {}'.format(X.element_size() * X.numel()))
    # print('size of Y: {}'.format(Y.element_size() * Y.numel()))
    X = X.view([1, -1])
    Y = Y.view([1, -1])
    pcc = ((X - X.mean())*(Y - Y.mean())).sum() / ((X - X.mean())*(X- X.mean())).sum().sqrt() / ((Y - Y.mean())*(Y- Y.mean())).sum().sqrt()    
    
    return pcc

def calculate_loss(sample, pos_scores, neg_scores, pos, neg, config):
    item_pop_train_dict = config['item_pop_train_dict']
    warm_item_train = config['warm_item_list']
    cold_item_train = config['cold_item_list']
    acc_w = config['weight']
    pop_w = 1 - acc_w
    """计算损失"""
    if sample == 'bpr':
        return -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean(), None
    elif sample in ['posneg']:
        acc_loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean() / 2
        pop_loss = -torch.log(1 - torch.tanh(pos_scores + neg_scores).abs()).mean() / 2
        return acc_loss * acc_w + pop_loss * pop_w, (acc_loss.item(), pop_loss.item())
    elif sample in ['pd']:
        pos_weight = torch.tensor([item_pop_train_dict[item.item()] for item in pos], device=pos.device)
        neg_weight = torch.tensor([item_pop_train_dict[item.item()] for item in neg], device=neg.device)
        m = nn.ELU()
        return -torch.log(torch.sigmoid((m(pos_scores) + 1.0) * pos_weight ** acc_w - (m(neg_scores) + 1.0) * neg_weight ** acc_w)).mean(), None
    # elif sample == 'ips':
    #     pos_weight = torch.tensor([1.0 / item_pop_train_dict[item.item()] for item in pos], device=pos.device)
    #     neg_weight = torch.tensor([1.0 / item_pop_train_dict[item.item()] for item in neg], device=neg.device)
    #     return -torch.log(torch.sigmoid(pos_scores) * pos_weight + torch.sigmoid(1 - neg_scores) * neg_weight).mean(), None
    elif sample == 'pearson':
        return -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean(), None
    elif sample == 'pearson_new':
        acc_loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
        pearson_loss = pcc_loss(pos, pos_scores, item_pop_train_dict)
        return acc_loss + pop_w * pearson_loss, (acc_loss.item(), pearson_loss.item())
    else:
        raise ValueError(f"Unsupported sample method: {sample}")

def calculate_regularization(model, user, pos, neg, reg_1, reg_2, batch_size):
    """计算正则化项"""
    user_emb = model.embed_user(user)
    pos_emb = model.embed_item(pos)
    neg_emb = model.embed_item(neg)
    reg = (reg_1 * (torch.norm(user_emb) ** 2) + reg_2 *(torch.norm(pos_emb) ** 2 + torch.norm(neg_emb) ** 2)) / 3 / batch_size
    return reg

def train_baseline_model(model, train_loader, train_set, config, device='cuda'):
    """训练函数"""
    item_pop_train_dict = config['item_pop_train_dict']
    warm_item_train = config['warm_item_list']
    cold_item_train = config['cold_item_list']
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    weight = config['weight']
    
    # 训练循环
    for epoch in range(config['epochs']):
        print(f'Epoch {epoch + 1}/{config["epochs"]}')
        model.train()
        model.to(device)
        
        if config['debias_method'] == 'cnif':
            config['burnin'] = 'yes'
        # 动态选择采样方法
        sample = 'bpr' if epoch < 10 and config['burnin'] == 'yes' else config['debias_method']

        # 遍历数据加载器
        if sample == 'cnif':
            for i in range(10):
                model.zero_grad()
                # users = np.array(0, user_nums, sample_user)
                # if sample == 'cgf_topk':
                #     print('train items...')
                #     users = np.array(range(train_user_nums))
                #     items = np.array(range(train_item_nums))
                users = np.array(range(config['user_num']))
                items = np.array(range(config['item_num']))
                cgf_loss = cgf_topk_loss(model, users, items, config['warm_item_list'], config['cold_item_list'], config['topk'])   
                loss = weight * ((cgf_loss - 0.3430)**2)
                print(f'cgf_loss of epoch : {cgf_loss.item()}') 
                print(f'loss of epoch : {loss.item()}')  
                loss.backward()
                optimizer.step()
        else:
            count = 0
            for user, pos1, neg1 in tqdm(train_loader, desc='Training'):
                # 将数据移动到设备
                user = user.to(device)
                pos = pos1.to(device)
                neg = neg1.to(device)

                # 清零梯度
                optimizer.zero_grad()

                # 计算 pos_score 和 neg_score
                if config['algo_name'] == 'mf':
                    pos_scores = model(user, pos)
                    neg_scores = model(user, neg)
                elif config['algo_name'] == 'lightgcn':
                    embed_user = model.embed_user(user)
                    embed_pos = model.embed_item(pos)
                    embed_neg = model.embed_item(neg)
                    pos_scores = (embed_user * embed_pos).sum(dim=-1)
                    neg_scores = (embed_user * embed_neg).sum(dim=-1)

                # 计算损失
                loss, extra_loss_info = calculate_loss(sample, pos_scores, neg_scores, pos, neg, config)

                # 添加正则化项
                if config['add_reg'] == 'yes':
                    loss += calculate_regularization(model, user, pos, neg, config['reg_1'], config['reg_2'], config['batch_size'])

                # 反向传播和优化
                loss.backward()
                optimizer.step()

                # 打印损失信息
                if count % 1000 == 0:
                    if extra_loss_info:
                        print(f'Loss: {loss.item()}, Extra acc Loss: {extra_loss_info[0]}, Extra pop Loss: {extra_loss_info[1]}')
                count = count + 1
            if sample == 'pearson':
                pcc = pcc_train(model, train_set, item_pop_train_dict, config) 
                loss = weight * (pcc**2)
                print(f'pcc loss of epoch : {pcc.item()}')
                print(f'loss of epoch : {loss.item()}')
                loss.backward()
                optimizer.step()

    return model

In [31]:
item_pop_train = pd.read_csv(saved_data_path + 'item_pop_train.csv',index_col=0)
item_pop_train_dict = dict(list(zip(item_pop_train.item, item_pop_train.train_counts)))

In [40]:
warm_item_list = np.load(saved_data_path + 'warm_item_list.npy')
cold_item_list = np.load(saved_data_path + 'cold_item_list.npy')
config['warm_item_list'] = warm_item_list
config['cold_item_list'] = cold_item_list
config['item_pop_train_dict'] = item_pop_train_dict

In [41]:
len(warm_item_list)

4457

In [42]:
len(cold_item_list)

9801

In [14]:
model

MF(
  (embed_user): Embedding(1411, 100)
  (embed_item): Embedding(3317, 100)
)

In [15]:
model = train_baseline_model(model, train_loader, train_total_set, config, device='cuda')

Epoch 1/20


Training: 100%|██████████| 2875/2875 [00:14<00:00, 205.24it/s]


Epoch 2/20


Training: 100%|██████████| 2875/2875 [00:14<00:00, 201.59it/s]


Epoch 3/20


...

Epoch 9/20


Training: 100%|██████████| 2875/2875 [00:14<00:00, 194.17it/s]


Epoch 10/20


Training: 100%|██████████| 2875/2875 [00:14<00:00, 204.99it/s]


Epoch 11/20
cgf_loss of epoch : 0.46597808599472046
loss of epoch : 0.15123611688613892
cgf_loss of epoch : 0.4641956388950348
loss of epoch : 0.14688384532928467
cgf_loss of epoch : 0.46067672967910767
loss of epoch : 0.1384781450033188
cgf_loss of epoch : 0.4559134244918823
loss of epoch : 0.1274944245815277
cgf_loss of epoch : 0.44987350702285767
loss of epoch : 0.11421947926282883
cgf_loss of epoch : 0.4430917203426361
loss of epoch : 0.10018353164196014
cgf_loss of epoch : 0.43525826930999756
loss of epoch : 0.08511588722467422
cgf_loss of epoch : 0.4271950423717499
loss of epoch : 0.07088805735111237
cgf_loss of epoch : 0.4188545346260071
loss of epoch : 0.057539112865924835
cgf_loss of epoch : 0.41012030839920044
loss of epoch : 0.045051366090774536
Epoch 12/20
cgf_loss of epoch : 0.40156492590904236
loss of epoch : 0.03429851308465004
cgf_loss of epoch : 0.39288848638534546
loss of epoch : 0.024888616055250168
cgf_loss of epoch : 0.38400405645370483
loss of epoch : 0.0168133303

In [16]:
debias_method = config['debias_method']
epochs = config['epochs']
model_pth =  f"{config['debias_method']}_{config['weight']}_{config['epochs']}.pth"
model_pth

'cnif_10_20.pth'

In [17]:
torch.save(model, saved_model_path + model_pth)

In [1]:
if 0.0 == 0:
    print('yes')

yes


In [ ]:
# method : bpr 、 ips no weight
# posneg: weight = 0.9
# pd: 0.01
# [0.01, 0.05, 0.1, 0.15, 0.2]
# pearson: 0.5
# [0.1, 0.01, 0.001, 0.0001] 0.5
# cnif: 10.0
# 10.0 100.0

In [ ]:
python drs_train.py --save=2 --dataset='kuairec_big' --algo_name='lightgcn' --debias_method='backbone' --weight=1 --gpu=0 --batch_size=8192

In [ ]:
python drs_train.py --save=2 --dataset='steam' --algo_name='lightgcn' --debias_method='backbone' --weight=0.01 --gpu=0

In [ ]:
python drs_train.py --save=2 --dataset='kuairec_big' --algo_name='lightgcn' --debias_method='cnif' --weight=10.0 --gpu=1

In [ ]:
python drs_train.py --save=2 --dataset='kuairec' --algo_name='lightgcn' --debias_method='posneg' --weight=0.9 --gpu=1

In [ ]:
python drs-fairagent.py --save=2 --dataset='kuairec' --algo_name='mf' --alpha=3 --beta=0.1 --gama=0.1 --gpu=0 --batch_size=32768

In [ ]:
python drs-fairagent-final.py --save=2 --dataset='steam' --algo_name='mf' --alpha=2 --beta=0.5 --gama=0.1 --gpu=1

In [ ]:
python train_aldi.py --save=1 --dataset='kuairec' --algo_name='mf' --debias_method='backbone' --gpu=1 